In [1]:

# Import pandas library
import pandas as pd

In [3]:
# initialize list
data = [['A'], ['B'], ['C']]
 
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Column1'])
 
# initialize list of lists
data2 = [['E'], ['F'], ['G']]
 
# Create the pandas DataFrame
df2 = pd.DataFrame(data2, columns = ['Column2'])

# initialize list of lists
data3 = [['X'], ['Y'], ['Z']]
 
# Create the pandas DataFrame
df3 = pd.DataFrame(data3, columns = ['Column3'])

Essentially, we need to cross join DF1, DF2, DF3

In [4]:
# Now to perform cross join, we will create
# a key column in both the DataFrames to 
# merge on that key.
df['key'] = 1
df2['key'] = 1
df3['key'] = 1

In [5]:
# to obtain the cross join we will merge on 
# the key - need to keep the key so we can merge with DF3
result = pd.merge(df, df2, on ='key')
result

,Column1,key,Column2
0,A,1,E
1,A,1,F
2,A,1,G
3,B,1,E
4,B,1,F
5,B,1,G
6,C,1,E
7,C,1,F
8,C,1,G


In [8]:
# Last merge result with DF3 on the key and now we drop it.
result2 = pd.merge(result, df3, on ='key').drop("key", 1)

#add color column - we don't want color column to be cross joined with the other values - just added as a column at the end. 
result2['color'] = '#ffc940'
result2['options'] = ""
result2['label'] = result2['Column1'] + result2['Column2'] + result2['Column3']
  
result2.head(20)

/var/folders/zr/p3dmj0t57nq_9sd5gmyrfgy40000gq/T/ipykernel_29567/2188135057.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  result2 = pd.merge(result, df3, on ='key').drop("key", 1)


,Column1,Column2,Column3,color,options,label
0,A,E,X,#ffc940,,AEX
1,A,E,Y,#ffc940,,AEY
2,A,E,Z,#ffc940,,AEZ
3,A,F,X,#ffc940,,AFX
4,A,F,Y,#ffc940,,AFY
5,A,F,Z,#ffc940,,AFZ
6,A,G,X,#ffc940,,AGX
7,A,G,Y,#ffc940,,AGY
8,A,G,Z,#ffc940,,AGZ
9,B,E,X,#ffc940,,BEX


Now we create an `y` column with our CQL expression in it

In [15]:
# "y": "sum{change:depart if department=\"	\" && title=\"	\" && payrollName=\" 	\"

#concat `CQL` expression:
result2['y'] = '\"sum{change:depart if department=\"' + result2['Column1'] + '\" && title=\"' + result2['Column2'] + '\" && payrollName=\"' + result2['Column3'] + '\"' + "}"

#change data type in this whole column as string:
result2['y'].astype('string')

#visual check to make sure it works
result2.head()

,Column1,Column2,Column3,color,options,label,y
0,A,E,X,#ffc940,,AEX,"""sum{change:depart if department=""A"" && title=..."
1,A,E,Y,#ffc940,,AEY,"""sum{change:depart if department=""A"" && title=..."
2,A,E,Z,#ffc940,,AEZ,"""sum{change:depart if department=""A"" && title=..."
3,A,F,X,#ffc940,,AFX,"""sum{change:depart if department=""A"" && title=..."
4,A,F,Y,#ffc940,,AFY,"""sum{change:depart if department=""A"" && title=..."


In [16]:
result2['y'][4]

'"sum{change:depart if department="A" && title="F" && payrollName="Y"}'

In [17]:
#send it to JSON Town! 
result2.to_json('result2.json', orient='records')

OSError: [Errno 30] Read-only file system: 'result2.json'